In [1]:
#Restart Run Time After This Version Of SKlearn
!pip install plotly-express
!pip install pycaret[full]
!pip install pandas-profiling==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.2 MB/s 
     |████████████████████████████████| 262 kB 47.8 MB/s 
     |████████████████████████████████| 167 kB 55.9 MB/s 
     |████████████████████████████████| 6.8 MB 63.4 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 1.7 MB 41.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 6.8 MB/s 
     |████████████████████████████████| 1.3 MB 46.7 MB/s 
     |████████████████████████████████| 2.0 MB 43.8 MB/s 
     |████████████████████████████████| 120 kB 60.9 MB/s 
     |████████████████████████████████| 17.8 MB 515 kB/s 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 261 kB 5.1 MB/s 
     |████████████████████████████████| 303 kB 52.4 MB/s 
     |████████████████████████████████| 3.1 MB 51.4 MB/s 
  Attempting uninstall: tangled-up-in-unicode
    Found existing installation: tangled-up-in-unicode 0.2.0
    Uninstalling tangled-up-in-unicode-0.2.0:
      Successfully uninstalled tangled-up-in-unicode-0.2.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 3.2.0
    Uninstalling pandas-profiling-3.2.0:
      Successfully uninstalled pandas-profiling-3.2.0
ERRO

In [1]:
import pandas as pd
import matplotlib
import matplotlib_inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import pycaret
from pycaret.regression import *

In [46]:
data = pd.read_excel("/content/DAILY ER.xlsx")
data

,DATE,COUNT
0,01-Jun2021,125
1,02-Jun2021,149
2,03-Jun2021,121
3,04-Jun2021,95
4,05-Jun2021,121
...,...,...
352,19-May2022,166
353,20-May2022,125
354,21-May2022,84
355,20-May2022,125


In [47]:
data['DATE'] = pd.to_datetime(data['DATE'])
data.head(30)
data['MA30'] = data['COUNT'].rolling(30).mean()

In [49]:
# plot the data and MA
import plotly.express as px
fig = px.line(data, x="DATE", y=["COUNT", "MA30"], template = 'plotly_dark')
fig.show()
# extract month and year from dates**
data['Month'] = [i.month for i in data['DATE']]
data['Year'] = [i.year for i in data['DATE']]

In [50]:
# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)

# drop unnecessary columns and re-arrange
data.drop(['DATE', 'MA30'], axis=1, inplace=True)
data = data[['Series', 'Year', 'Month','COUNT']]

In [51]:
# check the head of the dataset**
data.head(5)

,Series,Year,Month,COUNT
0,1,2021,6,125
1,2,2021,6,149
2,3,2021,6,121
3,4,2021,6,95
4,5,2021,6,121


In [52]:
# split data into train-test set
train = data[data['Series'] < 300]
test = data[data['Series'] >= 300]

In [53]:
s = setup(data = train, test_data = test,
          target = 'COUNT',
          fold_strategy = 'timeseries',
          numeric_features = ['Year', 'Series'],
          fold = 3, 
          transform_target = True,
          remove_outliers=False,
          session_id = 125)

,Description,Value
0,session_id,125
1,Target,COUNT
2,Original Data,"(299, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(299, 12)"


In [54]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ada,AdaBoost Regressor,32.3632,1719.9887,39.8521,-0.2022,0.2850,0.2505,0.0467
catboost,CatBoost Regressor,34.8689,1909.2752,41.4441,-0.2690,0.2971,0.2734,0.5400
lightgbm,Light Gradient Boosting Machine,34.9555,1907.0214,42.1405,-0.3806,0.2995,0.2648,0.0300
lr,Linear Regression,37.9145,2315.9738,46.0112,-0.9500,0.3395,0.2758,0.0200
rf,Random Forest Regressor,38.3664,2350.5599,44.7792,-0.4241,0.3168,0.3068,0.3533
dummy,Dummy Regressor,38.8699,2315.0752,47.9337,-1.0170,0.3268,0.2581,0.0200
llar,Lasso Least Angle Regression,38.8699,2315.0753,47.9337,-1.0170,0.3268,0.2581,0.0233
gbr,Gradient Boosting Regressor,39.4396,2512.6989,45.8719,-0.4805,0.3219,0.3144,0.0500
lar,Least Angle Regression,39.6200,2544.5315,47.7912,-1.1877,0.3595,0.2869,0.0200
xgboost,Extreme Gradient Boosting,39.8081,2616.5809,46.3799,-0.5017,0.3239,0.3179,0.3200


In [55]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,27.6791,1233.0199,35.1144,-0.9482,0.2656,0.1917


In [57]:
# generate predictions on the original dataset**
predictions = predict_model(best, data=data)



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,19.5233,614.2669,24.7844,0.5723,0.1849,0.1438


In [61]:
# add a date column in the dataset**
predictions['DATE'] = pd.date_range(start='2021-06-01', end = '2022-05-23')

In [62]:
# line plot**
fig = px.line(predictions, x='DATE', y=["COUNT", "Label"], template = 'presentation')
# add a vertical rectange for test-set separation**
fig.add_vrect(x0="2022-03-01", x1="2022-05-01", fillcolor="Red", opacity=0.25, line_width=1)

In [63]:
future_dates = pd.date_range(start = '2022-05-022', end = '2023-08-01')

future_df = pd.DataFrame()

future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates] 

ValueError: ignored